In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from tqdm import trange
plt.style.use(['science','grid'])

# import custom modules
import Pub
import PubCrawlFunctions as PCF
import Ant
import Logger
# import randomPubsInit

In [ ]:
# simulation paramters
tau0 = 1
alpha = 1
beta = 1
gamma = 1 
rho = 0.2

# simulation counters
time = 0
timeMax = int(60*12)            # 12 hours in minutes - 3pm to 3am 
iter = 0
maxIter = 1000
# population size of ants
popSize = 100

# velocity of an ant
velAnt = 1


In [ ]:
Pubs = PCF.initPubs('randomPubs.csv')
# init the pheromone matrix which is a 2D array with the size of the number of pubs
pheromoneMatrix = np.ones((len(Pubs), len(Pubs)))
pheromoneMatrix = pheromoneMatrix * tau0

# init the distance matrix D
distanceMatrix = np.zeros((len(Pubs), len(Pubs)))
for i in range(len(Pubs)):
    for j in range(i, len(Pubs)):
        distanceMatrix[i][j] = PCF.getDistance(Pubs[i], Pubs[j])
        #  print("i and j: ", i, j, " distance: ", distanceMatrix[i][j])

        distanceMatrix[j][i] = distanceMatrix[i][j]

# set the diagonal to 10e15
for i in range(len(Pubs)):
    distanceMatrix[i][i] = 10e15

# init the visibility matrix
visibilityMatrix = 1 / distanceMatrix

In [ ]:
# Ant colony

pathCollection = np.zeros((popSize, len(Pubs)))
pathLengthCollection = np.zeros((popSize, 1))

minimumPathLength = int(10e15)
minimumPath = np.zeros(len(Pubs))
bestAnt = None


# clear the log file
Logger.clearLog()


## Main algorithm

while(iter < maxIter):
    iter += 1

    # give heartbeat
    if iter % 100 == 0:
        print("Iteration: ", iter)

    # Generate paths
    for i in range(popSize):
        # create an Ant
        ant = Ant.Ant(velocity=1)
        path = PCF.generatePath(pheromoneMatrix, visibilityMatrix, alpha, beta, gamma, Pubs, ant)
        #pathLength = PCF.getPathLength(path, Pubs)
        pathLength = PCF.getPathDuration(ant)
        # pathDuration = PCF.getPathDuration(path, Pubs, ant)
        

        # update the minimal path
        if pathLength < minimumPathLength:
            minimumPathLength = pathLength
            minimumPath = path
            minimumPathTimeTrajectory = ant.timedPath
            bestAnt = ant

            # inform the user
            print('New minimum path found, in iteration: ', minimumPathLength, iter)
            print("Path: ", minimumPath)

            # log the new minimum path
            Logger.logBestPath(minimumPath, minimumPathTimeTrajectory, minimumPathLength)

            # if Plotting:
            #     # update scatter plot
            #     x = [Pubs[pubID].posX for pubID in minimumPath]
            #     y = [Pubs[pubID].posY for pubID in minimumPath]
            #     pubs_scatter.set_offsets(np.column_stack((x, y)))

            #     # update connection lines
            #     connection_lines.set_xdata(x)
            #     connection_lines.set_ydata(y)

            #     # update the pheromone matrix
            #     axs[1].imshow(pheromoneMatrix, cmap='hot', interpolation='nearest')

            #     # update the path length plot
            #     pathLengthPlot.set_xdata(np.append(pathLengthPlot.get_xdata(), iter*(popSize)+(i+1)))
            #     pathLengthPlot.set_ydata(np.append(pathLengthPlot.get_ydata(), minimumPathLength))
            #     axs[2].set_xlim(0, iter*(popSize)+(i+1))
            #     if minimumPathLength > LengthMax:
            #         LengthMax = minimumPathLength
            #         axs[2].set_ylim(0, int(minimumPathLength*1.1))


            #     # update the figure title to the minimal path length and iteration
            #     fig.suptitle('Pub Crawl, iteration: ' + str(iter) + ', path length: ' + str(minimumPathLength))

            #     plt.pause(0.01)





        pathCollection[i,:] = path
        pathLengthCollection[i] = pathLength

    # update the pheromone matrix
    deltaPheromoneMatrix = PCF.getDeltaPheromoneMatrix(pathCollection, pathLengthCollection)
    pheromoneMatrix = PCF.updatePheromoneMatrix(pheromoneMatrix, deltaPheromoneMatrix, rho)




In [ ]:
# inform the user over the best found ant
print("Best ant: ", bestAnt)
print("Best path: ", minimumPath)
print("Best path length: ", minimumPathLength)
print("Best path time trajectory: ", minimumPathTimeTrajectory)
